# DSPY
DSPy is a framework for algorithmically optimizing LM prompts and weights. DSPy can help you define your your tasks more accurately and can help you optimize your prompt for your sutaible use case.

Read More about it [here](https://dspy-docs.vercel.app/docs/intro)


Let's explore different components of DSPY with the help of the below examples.

## DSPY Components 

### Signatures 

They are the definition of input and output to your application/pipeline. They contain the definition of the task you want to solve along with inputs and outputs of your task. They can be defined in two ways. 

- Inline Signature 
- Class Based Signature 
    
[Reference](https://dspy-docs.vercel.app/docs/building-blocks/signatures)

### LM and RM Pipeline 

LM modules/Clients are the Language Models on which you want to base your pipeline on.(for eg. `gpt-3.5` ,`gpt-4` or any open source LLM). 

[Reference](https://dspy-docs.vercel.app/docs/building-blocks/language_models)

RM modules are optional and consist of a retrieval module that can provide necessary context to your LM for answer generation. 

[Reference](https://dspy-docs.vercel.app/docs/category/retrieval-model-clients)



### Metrics 

Metrics are the evaluation criteria for your pipeline. They can be used to evaluate the performance of your pipeline and optimize the pipeline based on your defined set of metrics.

[Reference](https://dspy-docs.vercel.app/docs/building-blocks/metrics)

### Optimizer/Teleprompter

It is an algorithm desgined to tune the prompts and signature of your LM models such that it can perform better on your defined metrics. 

[Reference](https://dspy-docs.vercel.app/docs/building-blocks/optimizers)


Let's understand how we can leverage DSPY to optimize our Prompts and generate better answers

In [16]:
import pandas as pd
import dspy
from fastembed import TextEmbedding
from qdrant_client import QdrantClient
from qdrant_client.http import models 
from tqdm import tqdm
import json
from typing import Optional,List,Union
from pydantic import BaseModel
from fastembed import TextEmbedding
from dsp.utils import dotdict
import os
from dotenv import load_dotenv
from qdrant_client import QdrantClient
import random

In [ ]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

### Load the Training and Testing Data

In [17]:
test_data = pd.read_csv('../data/QA/test.csv')
train_data = pd.read_csv('../data/QA/train.csv')
train_data.head()

question  \
0                                                                         What is Snord116?   
1                                            Are ultraconserved elements often transcribed?   
2                                                            List metalloenzyme inhibitors.   
3  Which protein phosphatase has been found to interact with the heat shock protein, HSP20?   
4                         Do DNA double-strand breaks play a causal role in carcinogenesis?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              contexts  \
0  ['Further analysis with array-CGH identified a mosaic 847\u2009kb deletion in 15q11-q13, including SNURF-SNRPN, the snoRNA gene clusters SNORD116 (HBII-85), SNORD115, (HBII-52), SNORD109 A and B (HBII-438A and B), SNORD64 (HBII-13), and NPAP1 (C15ORF2).', 'All three deletions included SNORD116, but only two encompassed parts of SNURF-SNRPN, implicating SNORD116 as the major contributor to the Prader-Willi phenotype. Our case adds further information about genotype-phenotype correlation and s...   
1  ['Starting from a genome-wide expression profiling, we demonstrate for the first time a functional link between oxygen deprivation and the modulation of long noncoding transcripts from ultraconserved regions, termed transcribed-ultraconserved regions (T-UCRs)', 'Our data gives a first glimpse of a novel functional hypoxic network comprising protein-coding transcripts and noncoding RNAs (ncRNAs) from the T-UCRs category', 'Highly conserved elements discovered in vertebrates are present in non...   
2  [' Clinically approved inhibitors were selected as well as several other reported metalloprotein inhibitors in order to represent a broad range of metal binding groups (MBGs), including hydroxamic acid, carboxylate, hydroxypyridinonate, thiol, and N-hydroxyurea functional groups.', 'A total of 21 different raltegravir-chelator derivative (RCD) compounds were prepared that differed only in the nature of the MBG. ', 'At least two compounds (RCD-4, RCD-5) containing a hydroxypyrone MBG were fou...   
3  [' Moreover, protein phosphatase-1 activity is regulated by two binding partners, inhibitor-1 and the small heat shock protein 20, Hsp20. Indeed, human genetic variants of inhibitor-1 (G147D) or Hsp20 (P20L) result in reduced binding and inhibition of protein phosphatase-1, suggesting aberrant enzymatic regulation in human carriers. ', 'Small heat shock protein 20 interacts with protein phosphatase-1 and enhances sarcoplasmic reticulum calcium cycling.', ' Hsp20 overexpression in intact anim...   
4  ['The DNA non-homologous end-joining repair gene XRCC6/Ku70 plays an important role in the repair of DNA double-strand breaks (DSBs) induced by both exogenous and endogenous DNA-damaging agents. Defects in overall DSB repair capacity can lead to genomic instability and carcinogenesis.', 'The tumor suppressor breast cancer susceptibility protein 1 (BRCA1) protects our cells from genomic instability in part by facilitating the efficient repair of DNA double-strand breaks (DSBs). BRCA1 promotes...   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

Each Question in the training data is assosiated with ground_truth label , we will use this to train our model and optimize the prompts. 

In [18]:
pd.set_option('display.max_colwidth', 500)
train_data[['question', 'contexts', 'ground_truth']]

question  \
0                                                                            What is Snord116?   
1                                               Are ultraconserved elements often transcribed?   
2                                                               List metalloenzyme inhibitors.   
3     Which protein phosphatase has been found to interact with the heat shock protein, HSP20?   
4                            Do DNA double-strand breaks play a causal role in carcinogenesis?   
...                                                                                        ...   
1654                      Is thyroid hormone therapy indicated in patients with heart failure?   
1655                                        Is protein Fbw7 a SCF type of E3 ubiquitin ligase?   
1656                                                         Is Annexin V an apoptotic marker?   
1657                             Which are the clinical characteristics of Tuberous Sclerosis?   
1658                      Is irritable bowel syndrome more common in women with endometriosis?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 contexts  \
0     ['Further analysis with array-CGH identified a mosaic 847\u2009kb deletion in 15q11-q13, including SNURF-SNRPN, the snoRNA gene clusters SNORD116 (HBII-85), SNORD115, (HBII-52), SNORD109 A and B (HBII-438A and B), SNORD64 (HBII-13), and NPAP1 (C15ORF2).', 'All three deletions included SNORD116, but only two encompassed parts of SNURF-SNRPN, implicating SNORD116 as the major contributor to the Prader-Willi phenotype. Our case adds further information about genotype-phenotype correlation and s...   
1     ['Starting from a genome-wide expression profiling, we demonstrate for the first time a functional link between oxygen deprivation and the modulation of long noncoding transcripts from ultraconserved regions, termed transcribed-ultraconserved regions (T-UCRs)', 'Our data gives a first glimpse of a novel functional hypoxic network comprising protein-coding transcripts and noncoding RNAs (ncRNAs) from the T-UCRs category', 'Highly conserved elements discovered in vertebrates are present in non...   
2     [' Clinically approved inhibitors were selected as well as several other reported metalloprotein inhibitors in order to represent a broad range of metal binding groups (MBGs), including hydroxamic acid, carboxylate, hydroxypyridinonate, thiol, and N-hydroxyurea functional groups.', 'A total of 21 different raltegravir-chelator derivative (RCD) compounds were prepared that differed only in the nature of the MBG. ', 'At least two compounds (RCD-4, RCD-5) containing a hydroxypyrone MBG were fou...   
3     [' Moreover, protein phosphatase-1 activity is regulated by two binding partners, inhibitor-1 and the small heat shock protein 20, Hsp20. Indeed, human genetic variants of inhibitor-1 (G147D) or Hsp20 (P20L) result in reduced binding and inhibition of protein phosphatase-1, suggesting aberrant enzymatic regulation in human carriers. ', 'Small heat shock protein 20 interacts with protein phosphatase-1 and enhances sarcoplasmic reticulum calcium cycling.', ' Hsp20 overexpression in intact anim...   
4     ['The DNA non-homologous end-joining repair gene XRCC6/Ku70 plays an important role in the repair of DNA double-strand breaks (DSBs) induced by both exogenous and endogenous DNA-damaging agents. Defects in overall DSB repair capacity can lead to genomic instability and carcinogenesis.', 'The tumor suppressor breast cancer susceptibility protein 1 (BRCA1) p

In [19]:
test_data

id  \
0      1   
1      2   
2      3   
3      4   
4      5   
..   ...   
706  707   
707  708   
708  709   
709  710   
710  711   

                                                                                                      question  
0             Which are the Proprotein Convertase Subtilisin Kexin 9 (PCSK9) inhibitors that are FDA approved?  
1                                                      What is the relationship between nucleosomes and exons?  
2                                                                   What is the name for anorexia in gymnasts?  
3                      How is Lambert-Eaton myasthenic syndrome (LEMS) associated with small cell lung cancer?  
4                                                       Is Fibroblast Growth Factor 23 a phosphaturic hormone?  
..                                                                                                         ...  
706  Is alternative splicing of apoptotic genes playing a role in the response to DNA or mitochondrial damage?  
707                            Which NADPH oxidase family member requires interaction with NOXO1 for function?  
708                   List genes that have been found mutated in CMT1A (Charcot-Marie-Tooth disease type 1 A).  
709                                                                                            What is BioASQ?  
710                                                                                          What is Snord116?  

[711 rows x 2 columns]

### Upload Contexts to Qdrant Vector Store

In [30]:
contexts = pd.read_csv('../data/QA/contexts.csv')
contexts.head()

text
0                                                                                                                                                                                                                                                                                                                                                 Both 7SL genes and Alu elements are transcribed by RNA polymerase III, and we show here that the internal 7SL promoter lies within the Alu-like part of the 7SL gene
1  We performed a comparative analysis in vitro and in vivo of the antitumor effects of three different antibodies targeting different epitopes of ErbB2: Herceptin (trastuzumab), 2C4 (pertuzumab) and Erb-hcAb (human anti-ErbB2-compact antibody), a novel fully human compact antibody produced in our laboratory. Herein, we demonstrate that the growth of both androgen-dependent and independent prostate cancer cells was efficiently inhibited by Erb-hcAb. The antitumor effects induced by Erb-hcAb on ...
2                                                                                                                                                                                                                                          The weight-reducing property of molindone, a recently introduced antipsychotic drug, was tested in 9 hospitalized chronic schizophrenic patients. There was an average weight loss of 7.6 kg after 3 months on molindone; most of the loss occurred during the first month.
3                                                                                                                                                                                                                                                                                                        Our study identifies a unique heterochromatin state marked by the presence of both H3.3 and H3K9me3, and establishes an important role for H3.3 in control of ERV retrotransposition in embryonic stem cells.
4                                                                                                                                                                                                                                                                                                Polyneuropathy, organomegaly, endocrinopathy, monoclonal gammopathy, and skin changes (POEMS) syndrome is an uncommon condition related to a paraneoplastic syndrome secondary to an underlying plasma cell disorder.

In [22]:
embedding_model = TextEmbedding("BAAI/bge-base-en-v1.5")
qdrant_client = QdrantClient(":memory:") # spin up a local instance if you require more advanced features
# qdrant_client = QdrantClient("http://localhost:6333") # uncomment if you want to use your local instance 
qdrant_client.recreate_collection('rag_contexts',vectors_config=models.VectorParams(size=768, distance=models.Distance.COSINE))
# Create and upload points to Qdrant
points = []
# contexts_sample = contexts.sample(100)
for idx, row in tqdm(contexts.iterrows(),total=contexts.shape[0]):
    point = models.PointStruct(
        id=idx,  # Use the dataframe index as the point ID
        vector=list(embedding_model.embed(row['text']))[0],  # Convert the embedding to a list
        payload={'id': idx , "text":row['text']}  # Use the label_text as the payload
    )
    points.append(point)
qdrant_client.upload_points(collection_name='rag_contexts', points=points)

text
0                                                                                                                                                                                                                                                                                                                                                 Both 7SL genes and Alu elements are transcribed by RNA polymerase III, and we show here that the internal 7SL promoter lies within the Alu-like part of the 7SL gene
1  We performed a comparative analysis in vitro and in vivo of the antitumor effects of three different antibodies targeting different epitopes of ErbB2: Herceptin (trastuzumab), 2C4 (pertuzumab) and Erb-hcAb (human anti-ErbB2-compact antibody), a novel fully human compact antibody produced in our laboratory. Herein, we demonstrate that the growth of both androgen-dependent and independent prostate cancer cells was efficiently inhibited by Erb-hcAb. The antitumor effects induced by Erb-hcAb on ...
2                                                                                                                                                                                                                                          The weight-reducing property of molindone, a recently introduced antipsychotic drug, was tested in 9 hospitalized chronic schizophrenic patients. There was an average weight loss of 7.6 kg after 3 months on molindone; most of the loss occurred during the first month.
3                                                                                                                                                                                                                                                                                                        Our study identifies a unique heterochromatin state marked by the presence of both H3.3 and H3K9me3, and establishes an important role for H3.3 in control of ERV retrotransposition in embryonic stem cells.
4                                                                                                                                                                                                                                                                                                Polyneuropathy, organomegaly, endocrinopathy, monoclonal gammopathy, and skin changes (POEMS) syndrome is an uncommon condition related to a paraneoplastic syndrome secondary to an underlying plasma cell disorder.

### Custom Retriever that searchs the contexts from Qdrant Vector Store. 

In [23]:
# use any embedding model 
def generate_embeddings(text):
    return list(embedding_model.embed(text))[0]


class QdrantRetriever(dspy.Retrieve):
    def __init__(self,qdrant_collection_name,qdrant_client,k=10):
        super().__init__(k=k)
        self.client = qdrant_client
        self.collection_name = qdrant_collection_name

    def forward(self, query,k:Optional[int]=10):
        # Generate embedding for the query
        query_embedding = generate_embeddings(query)
        search_results = self.client.search(
            collection_name=self.collection_name,
            query_vector=query_embedding,
            limit=k if k else self.k
        )
        passages = [result.payload['text'] for result in search_results]
        passages = [dotdict({"long_text": passage}) for passage in passages]
        return passages

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

In [25]:
turbo = dspy.OpenAI(model="gpt-3.5-turbo-0125",api_key=openai_api_key,max_tokens=1000)
qdrant_client = QdrantClient(url="http://localhost:6333")

rm = QdrantRetriever("rag_contexts",qdrant_client)

# configure dspy with a RM Model and and LM Model
dspy.settings.configure(lm=turbo,rm=rm)


In [26]:
sample = test_data['question'].iloc[0]
dspy.Retrieve(k=10)(sample).passages

['Two proprotein convertase subtilisin/kexin type 9 (PCSK9) inhibitors, evolocumab and alirocumab, have recently been approved by both the Food and Drug Administration (FDA) and the European Medicines Agency (EMA) for the treatment of hypercholesterolemia.',
 'Food and Drug Administration approved the first two proprotein convertase subtilisin/kexin type 9 (PCSK9) inhibitors, alirocumab (Praluent®; Sanofi/ Regeneron) and evolocumab (Repatha®; Amgen), for use in patients with heterozygous and homozygous familial hypercholesterolemia and for patients intolerant of statins or those with a major risk of cardiovascular disease (CVD) but unable to lower their LDL cholesterol (LDL-C) to optimal levels with statins and ezetimibe.',
 'In 2015 the U.S. Food and Drug Administration approved the first two proprotein convertase subtilisin/kexin type 9 (PCSK9) inhibitors, alirocumab (Praluent®; Sanofi/ Regeneron) and evolocumab (Repatha®; Amgen), for use in patients with heterozygous and homozygous 

### Signature Defination for Q/A System

In [ ]:
# Define Signatire for the QA system
class GenerateAnswer(dspy.Signature):
    """Answer questions based on the context."""
    
    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField()

### RM(Retrieval Model) Pipeline Creation

In [27]:

# Define a Custom RAG Pipeline
class RAG(dspy.Module):
    def __init__(self, collection_name= "rag_contexts",num_passages=10):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)


    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)



In [28]:
uncompiled_rag = RAG()

In [ ]:
uncompiled_rag(sample)

In [16]:
turbo.inspect_history(n=1)





Answer questions based on the context.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Context:
[1] «Two proprotein convertase subtilisin/kexin type 9 (PCSK9) inhibitors, evolocumab and alirocumab, have recently been approved by both the Food and Drug Administration (FDA) and the European Medicines Agency (EMA) for the treatment of hypercholesterolemia.»
[2] «Food and Drug Administration approved the first two proprotein convertase subtilisin/kexin type 9 (PCSK9) inhibitors, alirocumab (Praluent®; Sanofi/ Regeneron) and evolocumab (Repatha®; Amgen), for use in patients with heterozygous and homozygous familial hypercholesterolemia and for patients intolerant of statins or those with a major risk of cardiovascular disease (CVD) but unable to lower their LDL cholesterol (LDL-C) to optimal levels with statins and ezetimibe.»
[3] «In 201

### Metrics Defination and Assesment Signatures  

In [17]:
metricLM = dspy.OpenAI(model='gpt-4',api_key=openai_api_key,max_tokens=1000, model_type='chat')

# Signature for LLM assessments.
class Assess(dspy.Signature):
    """Assess the quality of an answer to a question."""
    context = dspy.InputField(desc="The context for answering the question.")
    assessed_question = dspy.InputField(desc="The evaluation criterion.")
    assessed_answer = dspy.InputField(desc="The answer to the question.")
    correct_answer = dspy.InputField(desc="The correct answer to the question.")
    assessment_answer = dspy.OutputField(desc="A rating between 1 and 5. Only output the rating and nothing else.")

def llm_metric(gold, pred, trace=None):
    predicted_answer = pred.answer
    gold_question = gold.question
    gold_answer = gold.answer

    detail = "Is the assessed answer detailed?"
    faithful = "Is the assessed text grounded in the context? Say no if it includes significant facts not in the context."
    correctness = f"Compare the given {predicted_answer} and {gold_answer} and assess how correct the answer is"

    with dspy.context(lm=metricLM):
        context = dspy.Retrieve(k=10)(gold_question).passages
        detail = dspy.ChainOfThought(Assess)(context="N/A", assessed_question=detail, assessed_answer=predicted_answer,correct_answer=gold_answer)
        faithful = dspy.ChainOfThought(Assess)(context=context, assessed_question=faithful, assessed_answer=predicted_answer,correct_answer=gold_answer)
        correctness = dspy.ChainOfThought(Assess)(context=context,assessed_question=correctness, assessed_answer=predicted_answer,correct_answer=gold_answer)

    print(f"Faithful: {faithful.assessment_answer}")
    print(f"Detail: {detail.assessment_answer}")
    print(f"Correctness: {correctness.assessment_answer}")
    
    total = float(detail.assessment_answer) + float(faithful.assessment_answer) + float(correctness.assessment_answer)
    return total / 5.0
    

Reference for the above is taken from below cited sources 
- [Reference_1](https://dspy-docs.vercel.app/docs/building-blocks/metrics#intermediate-using-ai-feedback-for-your-metric)
- [Reference_2](https://github.com/weaviate/recipes/blob/main/integrations/dspy/1.Getting-Started-with-RAG-in-DSPy.ipynb)

Let's format the data in a specific way how the DSPY modules are expecting and then use some of the data for training and evaluation. 

In [18]:
trainset_dspy = train_data.sample(frac=0.8)
valset_dspy = train_data.drop(trainset_dspy.index)

In [19]:
from ast import literal_eval
import dspy

def read_list_from_string(s):
    try:
        return literal_eval(s)
    except (ValueError, SyntaxError):
        return s.split() if isinstance(s, str) else []

def stringify_list_elements(lst):
    lst = read_list_from_string(lst)
    return " ".join(str(e) for e in lst)

trainset = [dspy.Example(question=row['question'],
                        #  contexts=stringify_list_elements(row['contexts']),
                         answer=stringify_list_elements(row['ground_truth']))
            .with_inputs("question") for i, row in trainset_dspy.iterrows()]

valset = [dspy.Example(question=row['question'],
                        # contexts=stringify_list_elements(row['contexts']),
                        answer=stringify_list_elements(row['ground_truth']))
           .with_inputs("question") for i, row in valset_dspy.iterrows()]

In [20]:
# For the purpose of demonstration let's keep it to 20. Remeber to use it wisely as the evaluation / training is all tied to API calls
devset = valset[:20]

In [ ]:
from dspy.evaluate.evaluate import Evaluate

evaluate = Evaluate(devset=devset, num_threads=8, display_progress=True, display_table=5)
uncompile_k_10 = RAG(num_passages=10) 
uncompiled_10_metrics = evaluate(uncompile_k_10, metric=llm_metric,return_all_scores=True,
        return_outputs=True)

In [50]:
def create_score_dataframe(eval_output):
    # Extract questions and answers from the examples
    questions = [ex[0].question for ex in eval_output]
    answers = [ex[1].answer for ex in eval_output]
    scores = [ex[2] for ex in eval_output]
    # Create a DataFrame with questions, answers, and scores
    score_dataframe = pd.DataFrame({'question': questions, 'predicted_answer': answers, 'score': scores})
    return score_dataframe


In [55]:
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 500)
eval_outs = uncompiled_10_metrics[1]
eval_outs_df = create_score_dataframe(eval_outs)
print(f"Mean Score for the devset is {eval_outs_df['score'].mean()}")
eval_outs_df

Mean Score for the devset is 2.0


question  \
0                                                                       Can protein coding exons originate from ALU sequences?   
1                                                                                               List metalloenzyme inhibitors.   
2                                                   Are long non coding RNAs as conserved in sequence as protein coding genes?   
3                                                                       Is Fibroblast Growth Factor 23 a phosphaturic hormone?   
4                                                                            Do Conserved noncoding elements act as enhancers?   
5                                What is the name of the stem loop present in the 3' end of genes encoding for selenoproteins?   
6                                                                               Are ultraconserved elements often transcribed?   
7                                                           What is being measured with an accelerometer in back pain patients   
8                                                         Can NXY-059 be used for treatment of acute ischemic stroke patients?   
9                                      Which proteins control the degradation of cryptic unstable transcripts (CUTs) in yeast?   
10                                                                                                            What are prions?   
11                                                             Are there Conserved Noncoding Elements (CNEs) in plant genomes?   
12                                                                                               Is CD99 encoded by MIC2 gene?   
13                                                                  Does HuR bind to the untranslated regions (UTRs) of mRNAs?   
14                                                                Is there any software for automated analysis of FISH images?   
15  The antibodies MK-3475 and CT-011 have shown promising results in treating malignancies. Which protein are they targeting?   
16                                                                                       What enzyme is inhibied by Opicapone?   
17                                                            Is amantadine effective for treatment of disorders conciousness?   
18                                                                     Which are the mains risk factors of metabolic syndrome?   
19                                 How does benralizumab's mechanism differ from other IL-5 biotherapeutics for severe asthma?   

                                                                                                                                                                                                                                                                                                                                                       predicted_answer  score  
0                                                                                                                                                                                                                                                                                                                                                                   Yes    2.2  
1                                                               VT-1129, VT-1161, hydroxamic acid, carboxylate, hydroxypyridinonate, thiol, N-hydroxyurea, BB-3497, 8-halo-4-(3-chloro-4-fluoro-phenylamino)-6-[(1H-[1,2,3]triazol-4-ylmethyl)-amino]-quinoline-3-carbonitriles, 2,2'-dipyridylamine (DPA), triazacyclononane (TACN), 8-hydroxyquinoline, thapsigargin.    1.8  
2                                                                                                                                                                                                                                                                                     No

The Mean Score of the Uncompiled RAG is 2.0. The Maximum Score possible is 3.0. Since we scaled the total score by 5. 

### Prompt from the Uncompiled Model

In [56]:
turbo.inspect_history(n=1)





Answer questions based on the context.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Context:
[1] «The metabolic syndrome is a complex association of several risk factors including insulin resistance, dyslipidemia, and essential hypertension.»
[2] «The metabolic syndrome is a constellation of risk factors including glucose dysregulation, central obesity, dyslipidemia, and hypertension.»
[3] «The metabolic syndrome is a clustering of risk factors which predispose an individual to cardiovascular morbidity and mortality. There is general consensus regarding the main components of the syndrome (glucose intolerance, obesity, raised blood pressure and dyslipidaemia [elevated triglycerides, low levels of high-density lipoprotein cholesterol])»
[4] «Metabolic syndrome is a constellation of interrelated risk factors of metabolic origin incl

In [57]:
# Lets check the Metrics LLM Prompt as well
metricLM.inspect_history(n=3)





Assess the quality of an answer to a question.

---

Follow the following format.

Context: The context for answering the question.

Assessed Question: The evaluation criterion.

Assessed Answer: The answer to the question.

Correct Answer: The correct answer to the question.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Context:
[1] «The metabolic syndrome is a complex association of several risk factors including insulin resistance, dyslipidemia, and essential hypertension.»
[2] «The metabolic syndrome is a constellation of risk factors including glucose dysregulation, central obesity, dyslipidemia, and hypertension.»
[3] «The metabolic syndrome is a clustering of risk factors which predispose an individual to cardiovascular morbidity and mortality. There is general consensus regarding the main components of the syndrome (glucose intolerance, ob

In [64]:
# Since 'trainset' is a list and doesn't have a 'sample' method, we will define a function to sample from it
def sample_from_list(lst, fraction):
    sample_size = int(len(lst) * fraction)
    return random.sample(lst, sample_size)

# Now we use the function to sample 2% if the dataset
trainset_truncated = sample_from_list(trainset, 0.02)
len(trainset_truncated)

26

### Optimizer : Bootstrap Random Search Optimization

In [65]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
teleprompter = BootstrapFewShotWithRandomSearch(metric=llm_metric, 
                                                max_bootstrapped_demos=2,
                                                max_labeled_demos=4, 
                                                max_rounds=1,
                                                num_candidate_programs=2,
                                                num_threads=8)

few_shot_bootstrap_compiled_rag = teleprompter.compile(uncompile_k_10, trainset=trainset_truncated)

Going to sample between 1 and 2 traces per predictor.
Will attempt to train 2 candidate sets.


Average Metric: 4.0 / 2  (200.0):   8%|▊         | 2/26 [00:28<04:42, 11.77s/it]

Faithful: 5
Detail: 1
Correctness: 5
Faithful: 5
Detail: 2
Correctness: 2


Average Metric: 6.7 / 3  (223.3):  12%|█▏        | 3/26 [00:29<02:33,  6.68s/it]

Faithful: 5
Detail: 4
Correctness: 4.5


...

Faithful: 5
Detail: 5
Correctness: 5
Faithful: 5
Detail: 2
Correctness: 3


Average Metric: 57.800000000000004 / 24  (240.8):  92%|█████████▏| 24/26 [01:06<00:04,  2.20s/it]

Faithful: 5
Detail: 3
Correctness: 3
Faithful: 5
Detail: 5
Correctness: 5


Average Metric: 62.400000000000006 / 26  (240.0): 100%|██████████| 26/26 [01:14<00:00,  2.87s/it]

Faithful: 4
Detail: 4
Correctness: 3
Faithful: 5
Detail: 3
Correctness: 4
Average Metric: 62.400000000000006 / 26  (240.0%)
Score: 240.0 for set: [4]
New best score: 240.0 for seed 1
Scores so far: [193.85, 228.08, 228.46, 223.85, 240.0]
Best score: 240.0
Average of max per entry across top 1 scores: 2.4000000000000004
Average of max per entry across top 2 scores: 2.6076923076923078
Average of max per entry across top 3 scores: 2.665384615384615
Average of max per entry across top 5 scores: 2.730769230769231
Average of max per entry across top 8 scores: 2.730769230769231
Average of max per entry across top 9999 scores: 2.730769230769231
5 candidate programs found.


In [66]:
# Let's check the prompt for this compiled model
turbo.inspect_history(n=1)





Answer questions based on the context.

---

Question: Are CD44 variants (CD44v) associated with poor prognosis of metastasis?
Answer: Yes, several isoforms (obtained by by usage of ten variant exons in various combinations) have been causally related to metastasis.

Question: Does triiodothyronine stimulate red blood cell sodium potassium pump?
Answer: An inverse correlation between this enzymatic action and free triiodothyronine (FT3) levels. The effect of triiodothyronine (T3) on Na+,K(+)-ATPase activity in red blood cells may be different in vivo and in vitro.

Question: What systems have been developed for the numbering of antibody residues?
Answer: The most prevalent antibody numbering systems are the Kabat system, the Chothia system as well as the IMGT numbering system.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Context:


You can notice how the prompt has somewhat become more specific in handling the examples and have also added extra instructions. Let's now evaluate on the `devset` we created and see how the model performs. 

In [70]:
few_shot_bootstrap_compiled_rag_evals = evaluate(few_shot_bootstrap_compiled_rag, metric=llm_metric, return_all_scores=True, return_outputs=True)

  0%|          | 0/20 [00:00<?, ?it/s]

Average Metric: 2.2 / 1  (220.0):   5%|▌         | 1/20 [00:13<04:23, 13.85s/it]

Faithful: 5
Detail: 3
Correctness: 3


Average Metric: 5.2 / 2  (260.0):  10%|█         | 2/20 [00:14<01:45,  5.88s/it]

Faithful: 5
Detail: 5
Correctness: 5


...

Average Metric: 39.3 / 17  (231.2):  85%|████████▌ | 17/20 [00:23<00:01,  1.70it/s]

Faithful: 5
Detail: 3
Correctness: 4


Average Metric: 45.3 / 20  (226.5): 100%|██████████| 20/20 [00:24<00:00,  1.21s/it]

Faithful: 5
Detail: 2
Correctness: 3
Faithful: 5
Detail: 2
Correctness: 3
Faithful: 5
Detail: 2
Correctness: 3
Average Metric: 45.3 / 20  (226.5%)



/Users/aarishalam/.pyenv/versions/3.11.0/envs/learn_ai_eng/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2.2' '2.0' '2.4' '2.2' '2.4']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


In [75]:
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 500)
evals_outs_df = create_score_dataframe(few_shot_bootstrap_compiled_rag_evals[1])
print(evals_outs_df['score'].mean())
evals_outs_df

2.265


question  \
0                                                                       Can protein coding exons originate from ALU sequences?   
1                                What is the name of the stem loop present in the 3' end of genes encoding for selenoproteins?   
2                                                                                               List metalloenzyme inhibitors.   
3                                      Which proteins control the degradation of cryptic unstable transcripts (CUTs) in yeast?   
4                                                           What is being measured with an accelerometer in back pain patients   
5                                                                               Are ultraconserved elements often transcribed?   
6                                                                       Is Fibroblast Growth Factor 23 a phosphaturic hormone?   
7                                                         Can NXY-059 be used for treatment of acute ischemic stroke patients?   
8                                                              Are there Conserved Noncoding Elements (CNEs) in plant genomes?   
9                                                   Are long non coding RNAs as conserved in sequence as protein coding genes?   
10                                                                                       What enzyme is inhibied by Opicapone?   
11                                 How does benralizumab's mechanism differ from other IL-5 biotherapeutics for severe asthma?   
12                                                                     Which are the mains risk factors of metabolic syndrome?   
13                                                                                               Is CD99 encoded by MIC2 gene?   
14                                                                           Do Conserved noncoding elements act as enhancers?   
15                                                                                                            What are prions?   
16                                                                  Does HuR bind to the untranslated regions (UTRs) of mRNAs?   
17                                                            Is amantadine effective for treatment of disorders conciousness?   
18                                                                Is there any software for automated analysis of FISH images?   
19  The antibodies MK-3475 and CT-011 have shown promising results in treating malignancies. Which protein are they targeting?   

                                                                                                                                                                                                                                                                                                                          predicted_answer  score  
0                                                                                                                                                                                                                                                                              Yes, protein coding exons can originate from ALU sequences.    2.2  
1                                                                                                                                                                                                           The stem loop present in the 3' end of genes encoding for selenoproteins is called the Sec insertion sequence (SECIS) element.    3.0  
2                 The metalloenzyme inhibitors mentioned in the context are VT-1129, VT-1161, the N-formyl-hydroxylamine derivative BB-3497, 8-halo-4-(3-chloro-4-fluoro-phenylamino)-6-[(1H-[1,2,3]triazol-4-ylmethyl)-amino]-quinoline-3-carbonitriles, 2,2'-dipyridylamine (DPA), triazacyclononane (TACN), and the 8-hydroxyquinoline.    2.0  
3                          

Notice How the Mean Score has increased to 2.265 from 2.0. So, the model is performing better on the devset now after being optimized. 

In [76]:
few_shot_bootstrap_compiled_rag(sample)

Prediction(
    context=['Two proprotein convertase subtilisin/kexin type 9 (PCSK9) inhibitors, evolocumab and alirocumab, have recently been approved by both the Food and Drug Administration (FDA) and the European Medicines Agency (EMA) for the treatment of hypercholesterolemia.', 'Food and Drug Administration approved the first two proprotein convertase subtilisin/kexin type 9 (PCSK9) inhibitors, alirocumab (Praluent®; Sanofi/ Regeneron) and evolocumab (Repatha®; Amgen), for use in patients with heterozygous and homozygous familial hypercholesterolemia and for patients intolerant of statins or those with a major risk of cardiovascular disease (CVD) but unable to lower their LDL cholesterol (LDL-C) to optimal levels with statins and ezetimibe.', 'In 2015 the U.S. Food and Drug Administration approved the first two proprotein convertase subtilisin/kexin type 9 (PCSK9) inhibitors, alirocumab (Praluent®; Sanofi/ Regeneron) and evolocumab (Repatha®; Amgen), for use in patients with hetero

### Signature Optmiizer

Optimizing Signature is also a way you can try to improve the performance of your model. You can either plug the above bootstrapped compiled model to this or you can even use the uncompiled model.

In [77]:

from dspy.teleprompt import BayesianSignatureOptimizer

llm_prompter = dspy.OpenAI(model='gpt-4', max_tokens=2000, model_type='chat')

teleprompter = BayesianSignatureOptimizer(task_model=dspy.settings.lm,
                                          metric=llm_metric,
                                          prompt_model=llm_prompter,
                                          n=5,
                                          verbose=False)
kwargs = dict(num_threads=8, display_progress=True, display_table=0)
signature_compiled_rag = teleprompter.compile(uncompile_k_10, devset=trainset_truncated,
                                         optuna_trials_num=3,
                                         max_bootstrapped_demos=4,
                                         max_labeled_demos=4,
                                         eval_kwargs=kwargs)

[WARNING] BayesianSignatureOptimizer has been deprecated and replaced with MIPRO.  BayesianSignatureOptimizer will be removed in a future release. 

Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:

- Task Model: 26 examples in dev set * 3 trials * # of LM calls in your program = (78 * # of LM calls in your program) task model calls
- Prompt Model: # data summarizer calls (max 10) + 5 * 1 lm calls in program = 15 prompt model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calc

  0%|          | 0/26 [00:00<?, ?it/s]

  4%|▍         | 1/26 [00:01<00:32,  1.28s/it]

Faithful: 5
Detail: 1
Correctness: 5


  8%|▊         | 2/26 [00:05<01:08,  2.86s/it]

...

Faithful: 5Faithful: 3
Detail: 2
Correctness: 1

Detail: 2
Correctness: 3
Faithful: 5
Detail: 5
Correctness: 4.5
Faithful: 5
Detail: 1
Correctness: 5


Average Metric: 50.39999999999999 / 26  (193.8): 100%|██████████| 26/26 [00:13<00:00,  1.86it/s] 

Faithful: 5
Detail: 1
Correctness: 2
Faithful: 2
Detail: 1
Correctness: 1
Average Metric: 50.39999999999999 / 26  (193.8%)



[I 2024-04-30 01:04:35,641] Trial 2 finished with value: 193.84999999999997 and parameters: {'6311971984_predictor_instruction': 0, '6311971984_predictor_demos': 0}. Best is trial 0 with value: 229.62.


Returning generate_answer = ChainOfThought(GenerateAnswer(context, question -> answer
    instructions='Answer questions based on the context.'
    context = Field(annotation=str required=True json_schema_extra={'desc': 'may contain relevant facts', '__dspy_field_type': 'input', 'prefix': 'Context:'})
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
)) from continue_program


In [79]:
signature_compiled_rag_eval = evaluate(signature_compiled_rag, metric=llm_metric,return_all_scores=True, return_outputs=True)

Faithful: 5
Detail: 5
Correctness: 5



Average Metric: 15.2 / 6  (253.3):  30%|███       | 6/20 [00:01<00:02,  6.31it/s]

Faithful: 5
Detail: 5
Correctness: 5
Faithful: 5
Detail: 2
Correctness: 4
Faithful: 5
Detail: 3
Correctness: 3
Faithful: 5
Detail: 3
Correctness: 4
Faithful: 5
Detail: 3
Correctness: 4
Faithful: 4
Detail: 3
Correctness: 3


Faithful: 5
Detail: 5
Correctness: 5
Faithful: 5
Detail: 4
Correctness: 3
Faithful: 5
Detail: 3
Correctness: 3


Average Metric: 24.799999999999997 / 10  (248.0):  45%|████▌     | 9/20 [00:01<00:01,  8.04it/s]

...

Faithful: 5
Detail: 3
Correctness: 4
Faithful: 5
Detail: 2
Correctness: 3
Faithful: 5
Detail: 2
Correctness: 3


Faithful: 5
Detail: 2
Correctness: 3
Faithful: 4
Detail: 4
Correctness: 3
Faithful: 5
Detail: 4
Correctness: 3


Average Metric: 46.2 / 20  (231.0): 100%|██████████| 20/20 [00:04<00:00,  4.15it/s]

Faithful: 5
Detail: 3
Correctness: 4
Faithful: 1
Detail: 2
Correctness: 3
Average Metric: 46.2 / 20  (231.0%)



/Users/aarishalam/.pyenv/versions/3.11.0/envs/learn_ai_eng/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2.2' '3.0' '2.2' '2.0' '2.4']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


In [82]:
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 500)

signature_compiled_rag_eval_evals_df = create_score_dataframe(signature_compiled_rag_eval[1])
print(signature_compiled_rag_eval_evals_df['score'].mean())
signature_compiled_rag_eval_evals_df

2.31


question  \
0                                                                                               List metalloenzyme inhibitors.   
1                                                           What is being measured with an accelerometer in back pain patients   
2                                      Which proteins control the degradation of cryptic unstable transcripts (CUTs) in yeast?   
3                                                                               Are ultraconserved elements often transcribed?   
4                                                                       Is Fibroblast Growth Factor 23 a phosphaturic hormone?   
5                                                   Are long non coding RNAs as conserved in sequence as protein coding genes?   
6                                                                       Can protein coding exons originate from ALU sequences?   
7                                What is the name of the stem loop present in the 3' end of genes encoding for selenoproteins?   
8                                                                                                             What are prions?   
9                                                                            Do Conserved noncoding elements act as enhancers?   
10                                                        Can NXY-059 be used for treatment of acute ischemic stroke patients?   
11                                                                Is there any software for automated analysis of FISH images?   
12                                                                                       What enzyme is inhibied by Opicapone?   
13                                                            Is amantadine effective for treatment of disorders conciousness?   
14  The antibodies MK-3475 and CT-011 have shown promising results in treating malignancies. Which protein are they targeting?   
15                                                                                               Is CD99 encoded by MIC2 gene?   
16                                 How does benralizumab's mechanism differ from other IL-5 biotherapeutics for severe asthma?   
17                                                                     Which are the mains risk factors of metabolic syndrome?   
18                                                                  Does HuR bind to the untranslated regions (UTRs) of mRNAs?   
19                                                             Are there Conserved Noncoding Elements (CNEs) in plant genomes?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       predicted_answer  \
0                                                        Metalloenzyme inhibitors include VT-1129, VT-1161, hydroxamate molecules, siderophores, the N-formyl-hydroxylamine derivative BB-3497, tissue inhibitor of metalloproteinases-1 (Timp-1), 8-halo-4-(3-chloro-4-fluoro-phenylamino)-6-[(1H-[1,2,3]triazol-4-ylmethyl)-amino]-quinoline-3-carbonitriles, 2,2'-dipyridylamine (DPA), triazacyclononane (TACN), 8-hydroxyquinoline, and the sarcoplasmic/endoplasmic Ca(2+)-ATPase (SERCA) inhibitor thapsigargin.   
1                                                                                                                                                                                                                                                                 In back pain patients, an accelerometer is used to measure various aspects of physical activity, including overall 

In comparision to the Uncompiled model the mean accuracy on the devset has increased to 2.3 from 2.0. 